In [3]:
import os,sys
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
os.listdir()
os.chdir('/content/drive/My Drive/')
sys.path.append('/content/drive/My Drive/')


Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [5]:
X=pd.read_csv("new.csv")
X

,category,headline,author,link,description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,5/26/2018
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,5/26/2018
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,5/26/2018
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,5/26/2018
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",5/26/2018
...,...,...,...,...,...,...
93625,POLITICS,Will Obama Be Lincoln or Everett?,"Greg Honan, ContributorResearch Assistant to D...",https://www.huffingtonpost.com/entry/will-obam...,"The scene is a familiar one: Gettysburg, Novem...",1/16/2015
93626,LATINO VOICES,New Arizona Republican Governor Begins Legacy ...,"DeeDee Garcia Blase, ContributorFounder, Natio...",https://www.huffingtonpost.com/entry/new-arizo...,"According to the Associated Press, ""Arizona be...",1/16/2015
93627,IMPACT,An Open Letter to a Rising Star,"Kevin M. Ryan, ContributorPresident and CEO, C...",https://www.huffingtonpost.com/entry/an-open-l...,While J. Lo wiped away her tears on national t...,1/16/2015
93628,WOMEN,"These Days, Is 'Love' Meaningless?","Alexandra Villarreal, ContributorJohn Jay Scho...",https://www.huffingtonpost.com/entry/these-day...,"As a concept, love has always fascinated me. T...",1/16/2015


In [6]:
X = X[pd.isna(X['headline'])==False]
X = X[pd.isna(X['description'])==False]

In [7]:
X.head(3)

,category,headline,author,link,description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,5/26/2018
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,5/26/2018
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,5/26/2018


In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93629 entries, 0 to 93629
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     93629 non-null  object
 1   headline     93629 non-null  object
 2   author       83959 non-null  object
 3   link         93629 non-null  object
 4   description  93629 non-null  object
 5   date         93629 non-null  object
dtypes: object(6)
memory usage: 5.0+ MB


In [9]:
# tfidf calculation
text_content = X['headline']
vector = TfidfVectorizer(max_df=0.3,         # drop words that occur in more than X percent of documents
                             #min_df=8,      # only use words that appear at least X times
                             stop_words='english', # remove stop words
                             lowercase=True, # Convert everything to lower case 
                             use_idf=True,   # Use idf
                             norm=u'l2',     # Normalization
                             smooth_idf=True # Prevents divide-by-zero errors
                            )
tfidf = vector.fit_transform(text_content)

In [10]:
# Request function : search the top_n articles from a request ( request = string)
def search(tfidf_matrix,model,request, top_n = 5):
    request_transform = model.transform([request])
    similarity = np.dot(request_transform,np.transpose(tfidf_matrix))
    x = np.array(similarity.toarray()[0])
    indices=np.argsort(x)[-5:][::-1]
    return indices

# Find similar : get the top_n articles similar to an article 
def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [index for index in related_docs_indices][0:top_n]    

# Print the result
def print_result(request_content,indices,X):
    print('\nsearch : ' + request_content)
    print('\nBest Results :')
    for i in indices:
        print('id = {0:5d} - headline = {1}'.format(i,X['headline'].loc[i]))

In [11]:
request = 'entertainment '
#request = text_content[0]

result = search(tfidf,vector, request, top_n = 5)
print_result(request,result,X)


search : entertainment 

Best Results :
id = 31957 - headline = Huff Post's Entertainment Instagram: What's New
id = 58646 - headline = Obama Urges Scrutiny Of Donald Trump's Record: 'This Is Not Entertainment'
id = 83842 - headline = Reporter Question Proves Trump 2016 Is An Entertainment Story
id = 81207 - headline = HuffPost Explains Why Trump Coverage Is Entertainment On 'Morning Joe'
id =  3789 - headline = And Just Like That, Gaming And Mainstream Entertainment Are As One


In [13]:
#Save objects on filesystem
pickle.dump(X, open('X', 'wb')) 
pickle.dump(vector, open('vector', 'wb')) 
pickle.dump(tfidf, open('tfidf', 'wb')) 

In [15]:
# load objects on filesystem
X = pickle.load(open('X', 'rb'))
vector = pickle.load(open('vector', 'rb'))
tfidf = pickle.load(open('tfidf', 'rb'))